<a href="https://colab.research.google.com/github/humannx2/Machine-Learning/blob/main/TextGenTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


Import datasets

In [3]:
from datasets import load_dataset
# dataset=load_dataset('wikitext', 'wikitext-2-raw-v1')
dataset=load_dataset("json", data_files="mydata.json")
raw_dataset=dataset['train']
print(dataset)
# print(dataset['train'])

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['achievement', 'date', 'details'],
        num_rows: 8
    })
})


Creating tokenizer and preprocessing data using a checkpoint

In [31]:
from transformers import GPT2TokenizerFast
checkpoint="gpt2"
tokenizer=GPT2TokenizerFast.from_pretrained(checkpoint)
# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

def token_function(dataset):
  tokenized_input=tokenizer(dataset['achievement'],
                            padding='max_length',
                            truncation=True,
                            return_attention_mask=True,
                            return_tensors='pt',) #creating tokens for the whole dataset(i.e multiple sentences)

  # Set labels to be the same as input_ids
  tokenized_input["labels"] = tokenized_input["input_ids"]
  return tokenized_input

# Apply tokenization to dataset
tokenized_dataset=raw_dataset.map(token_function,batched=True)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Data Collator and Model Declaration

In [32]:
from transformers import GPT2LMHeadModel, DataCollatorForLanguageModeling #since we're creating a text generating model
model=GPT2LMHeadModel.from_pretrained(checkpoint)

# Convert the dataset to PyTorch tensors
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False)
# # Add code to check for None values in the BatchEncoding before moving to device
# def custom_data_collator(features):
#     # Check if any feature contains None values
#     for feature in features:
#         if any(v is None for v in feature.values()):
#             print("Warning: Found None values in a feature. Skipping this batch.")
#             return None  # Skip this batch
#     # If no None values found, use the original data collator
#     return data_collator(features)



Traning Argument Trainer

In [33]:
from transformers import TrainingArguments, Trainer
import torch
import os

# # Set environment variable for CUDA memory management
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# torch.cuda.empty_cache()
# # Move model to GPU if available
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# model.to(device)

training_args=TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    per_device_train_batch_size=4,   # Batch size for training
    num_train_epochs=20,              # Number of training epochs
    weight_decay=0.01,
    gradient_accumulation_steps=8,
    fp16=True, #mixed precision training
)

In [34]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
# Train the model
trainer.train()

# Save the model and tokenizer
trainer.save_model("./model")
tokenizer.save_pretrained("./model")

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss


('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.json',
 './model/merges.txt',
 './model/added_tokens.json',
 './model/tokenizer.json')

working with the model

In [35]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the trained model and tokenizer
model_path = "./model"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)


Text Generation

In [40]:
# Define the input prompt
input_text = "How to become an Entrepreneur?"

# Tokenize the input
# inputs = tokenizer.encode(input_text, return_tensors="pt")
# Tokenize the input with attention masks
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=128,
    return_attention_mask=True
)

input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

# # Generate output
# outputs = model.generate(
#     inputs,
#     max_length=100,
#     num_return_sequences=1,
#     no_repeat_ngram_size=2,
#     repetition_penalty=1.5,
#     top_k=50,
#     top_p=0.95,
#     temperature=0.7,
#     do_sample=True,
#     early_stopping=True
# )


# Generate output using the model with attention masks
outputs = model.generate(
    input_ids,
    attention_mask=attention_mask,  # Pass attention mask
    max_length=100,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    do_sample=True,
    early_stopping=True
)

# Decode the generated output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How to become an Entrepreneur?
This post outlines how I became a successful entrepreneur in the UK. You can find my blog at www/howtostartup-itself or on LinkedIn here. Follow me @LizYelp and Facebook Page for more articles like this one!
